<a href="https://colab.research.google.com/github/fertorresfs/Analyzing-Video-with-OpenCV-and-NumPy/blob/master/BERT_Quora_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

# **Trabalho apresentado a disciplina SCC5871 - Algoritmos de Aprendizado de Máquina**

Grupo 6

11919610 - Fernando Torres Ferreira da Silva


### Download e Instalação do kaggle
Tutorial: https://www.kaggle.com/general/74235

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
 #Check if everything is OK!
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              234KB  2021-08-01 09:24:52          10057  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           6145  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           2294  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2534  
fatiimaezzahra/famous-iconic-women        

### Download do Dataset

**Data:** Quora Question Pairs - https://www.kaggle.com/c/quora-question-pairs/data

In [ ]:
!kaggle competitions download -c quora-question-pairs
!unzip train.csv.zip -d train

100% 173M/173M [00:01<00:00, 107MB/s] 
100% 173M/173M [00:01<00:00, 96.0MB/s]
 43% 9.00M/21.2M [00:00<00:00, 25.4MB/s]
100% 21.2M/21.2M [00:00<00:00, 43.0MB/s]
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/4.95M [00:00<?, ?B/s]
100% 4.95M/4.95M [00:00<00:00, 44.9MB/s]
Archive:  train.csv.zip
  inflating: train/train.csv         


# **Requirements**

### Import Libs

In [ ]:
# Importando bibliotecas
import pandas as pd
!pip install ktrain
import ktrain
from ktrain import text

     |████████████████████████████████| 25.3 MB 96 kB/s 
     |████████████████████████████████| 6.8 MB 42.3 MB/s 
     |████████████████████████████████| 981 kB 48.2 MB/s 
     |████████████████████████████████| 263 kB 47.9 MB/s 
     |████████████████████████████████| 1.9 MB 44.2 MB/s 
     |████████████████████████████████| 1.2 MB 49.7 MB/s 
     |████████████████████████████████| 468 kB 45.2 MB/s 
     |████████████████████████████████| 3.3 MB 46.4 MB/s 
     |████████████████████████████████| 895 kB 29.5 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.2-py3-none-any.whl size=25283087 sha256=edadd16991d53f1d3eca23cecda4b6363c6ac4e128f39c15a5bd0992009d6d2e
  Stored in directory: /root/.cache/pip/wheels/88/be/4a/971c83a380a40f12e877f643ca1b94dc65f528f94c88dbcff7
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=5ab4737fc64046cee475d93f0973148dcaa9638714598b4a22e985b5cca7d695
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

In [ ]:
dataset = pd.read_csv('/content/train/train.csv')
dataset = dataset.dropna()
dataset

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


# Reduzindo e dividindo em conjunto de treino e teste

In [ ]:
dataset_size = 200000
train_size = dataset_size * 0.3 #30% para treino
# test_size = dataset_size - train_size

# Seleciona os primeiros 100k diferentes
dif = dataset.loc[dataset.is_duplicate == 0]
dif_part = dif.iloc[:int(dataset_size/2),:]

train_dif_part = dif_part.iloc[:int(train_size/2),:]
test_dif_part = dif_part.iloc[int(train_size/2):,:]

# Seleciona os primeiros 100k iguais
equal = dataset.loc[dataset.is_duplicate == 1]
equal_part = equal.iloc[:int(dataset_size/2),:]

train_equal_part = equal_part.iloc[:int(train_size/2),:]
test_equal_part = equal_part.iloc[int(train_size/2):,:]

# Concatena os dois conjuntos
train_df = pd.concat([train_dif_part,train_equal_part])
test_df = pd.concat([test_dif_part,test_equal_part])

In [ ]:
x_train = train_df[['question1', 'question2']].values
y_train = train_df['is_duplicate'].values
x_test = test_df[['question1', 'question2']].values
y_test = test_df['is_duplicate'].values

# # IMPORTANT: data format for sentence pair classification is list of tuples of form (str, str)
x_train = list(map(tuple, x_train))
x_test = list(map(tuple, x_test))

y_test[36252]

0

In [ ]:
print(x_train[0])
print(y_train[0])

('What is the step by step guide to invest in share market in india?', 'What is the step by step guide to invest in share market?')
0


In [ ]:
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=128, class_names=['non-duplicate', 'duplicate'])
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32) # lower bs if OOM occurs
#testes devem ser menores que 5, pois segundo o prof. são obtidos os melhores resultados   
learner.fit_onecycle(5e-5, 3)
#learner.fit_onecycle(4e-4, 3)
#learner.fit_onecycle(3e-4, 3)
#learnet.fit_onecycle(1e-3, 3)


preprocessing train...
language: en
sentence pairs detected


Is Multi-Label? False
preprocessing test...
language: en
sentence pairs detected





begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
 508/1875 [=======>......................] - ETA: 17:21:26 - loss: 0.4889 - accuracy: 0.7569

In [ ]:
learner.validate(class_names=t.get_classes())

------- 5e-5, 3 ------ (RESULTADOS DO ONECYCLE)
begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
1875/1875 [==============================] - 4853s 3s/step - loss: 0.4045 - accuracy: 0.8132 - val_loss: 0.3263 - val_accuracy: 0.8612
Epoch 2/3
1875/1875 [==============================] - 4863s 3s/step - loss: 0.2737 - accuracy: 0.8885 - val_loss: 0.2992 - val_accuracy: 0.8748
Epoch 3/3
1875/1875 [==============================] - 4871s 3s/step - loss: 0.1285 - accuracy: 0.9544 - val_loss: 0.3519 - val_accuracy: 0.8813
(RESULTADOS VAlIDATE)

---------------------------------------------------------------------------
learner.fit_onecycle(5e-4, 3)
begin training using onecycle policy with max lr of 0.0005...
Epoch 1/3
1875/1875 [==============================] - 2755s 1s/step - loss: 0.5757 - accuracy: 0.6462 - val_loss: 0.6940 - val_accuracy: 0.5000
Epoch 2/3
1875/1875 [==============================] - 2752s 1s/step - loss: 0.6953 - accuracy: 0.4990 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/3
1875/1875 [==============================] - 2740s 1s/step - loss: 0.6935 - accuracy: 0.5007 - val_loss: 0.6932 - val_accuracy: 0.5000
<tensorflow.python.keras.callbacks.History at 0x7f9e51281b70>

              precision    recall  f1-score   support

non-duplicate       0.00      0.00      0.00     70000
    duplicate       0.50      1.00      0.67     70000

     accuracy                           0.50    140000
    macro avg       0.25      0.50      0.33    140000
 weighted avg       0.25      0.50      0.33    140000

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
array([[    0, 70000],
       [    0, 70000]])
---------------------------------------------------------------------------
       begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
1875/1875 [==============================] - 2642s 1s/step - loss: 0.3981 - accuracy: 0.8178 - val_loss: 0.3220 - val_accuracy: 0.8608
Epoch 2/3
1875/1875 [==============================] - 2647s 1s/step - loss: 0.2785 - accuracy: 0.8860 - val_loss: 0.3055 - val_accuracy: 0.8705
Epoch 3/3
1875/1875 [==============================] - 2652s 1s/step - loss: 0.1325 - accuracy: 0.9543 - val_loss: 0.3516 - val_accuracy: 0.8796

<tensorflow.python.keras.callbacks.History at 0x7f859126fc88>

                precision    recall  f1-score   support

non-duplicate       0.89      0.87      0.88     70000
    duplicate       0.87      0.89      0.88     70000

     accuracy                           0.88    140000
    macro avg       0.88      0.88      0.88    140000
 weighted avg       0.88      0.88      0.88    140000

array([[60604,  9396],
       [ 7454, 62546]])



# **Plotando a perda do modelo**

In [ ]:
learner.plot()

In [ ]:
predictor = ktrain.get_predictor(learner.model, t)

In [ ]:
y_test[69980:70010]

In [ ]:
positive = x_test[69980]
negative = x_test[70010]

In [ ]:
print('Duplicate:\n%s' %(positive,))

In [ ]:
print('Non-Duplicate:\n%s' %(negative,))

In [ ]:
predictor.predict(positive)

In [ ]:
predictor.predict(negative)

In [ ]:
predictor.predict([positive, negative])

In [ ]:
predictor.save('/tmp/mrpc_model')

In [ ]:
p = ktrain.load_predictor('/tmp/mrpc_model')

In [ ]:
p.predict(positive)